<a href="https://colab.research.google.com/github/zhh25/Titanic/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os
path_root = '/content/drive/MyDrive/titanic'
train = pd.read_csv(os.path.join(path_root, 'data/train.csv'))
test =  pd.read_csv(os.path.join(path_root, 'data/test.csv'))
sample_submission =  pd.read_csv(os.path.join(path_root, 'data/sample_submission.csv'))
titanic = train.drop('Survived', axis = 1)
titanic_label = train['Survived'].copy()

# Transformation Pipeline




In [ ]:
exec(open('/content/drive/MyDrive/Colab Notebooks/titanic_data_pipeline.py' ).read())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# XGBoost

In [ ]:
! pip install --upgrade xgboost
import xgboost as xgb
xgb.__version__

In [ ]:
best_xgb = xgb.Booster()
best_xgb.load_model(os.path.join(path_root, 'models/best_xgb.json'))

In [ ]:
test_prepared_xgb =  xgb.DMatrix(test_prepared)
prob_xgb = best_xgb.predict(test_prepared_xgb) 

# Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
#! pip install keras_tuner
# import keras_tuner as kt
from tensorflow.keras.regularizers import l2

In [ ]:
best_nn = tf.keras.models.load_model(os.path.join(path_root, 'models/best_nn'))

In [ ]:
prob_nn = best_nn.predict(test_prepared) 

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

import joblib

In [ ]:
best_rf = joblib.load(os.path.join(path_root, 'models/best_rf.sav'))

In [ ]:
prob_rf = best_rf.predict_proba(test_prepared)[:, 1]

# Ensemble

In [ ]:
probs = np.concatenate((np.reshape(prob_xgb, (418, 1)), 
                prob_nn, 
                np.reshape(prob_rf, (418, 1))),
                axis = 1
               )

In [ ]:
probs.mean(axis = 1)

array([0.08891816, 0.42362518, 0.12160552, 0.12913846, 0.54060752,
       0.13883211, 0.53357327, 0.15008351, 0.69052859, 0.11533474,
       0.13276709, 0.18555168, 0.92320954, 0.11616024, 0.88866428,
       0.84771109, 0.12775572, 0.16531274, 0.43643621, 0.52146295,
       0.23131407, 0.63094009, 0.8392084 , 0.30278775, 0.7327266 ,
       0.08963205, 0.90970971, 0.16290349, 0.33683305, 0.15253576,
       0.11870842, 0.13363124, 0.59115686, 0.51      , 0.44348022,
       0.17231092, 0.43089454, 0.43526509, 0.1174192 , 0.37544953,
       0.12526507, 0.54408031, 0.08565938, 0.79553618, 0.88869123,
       0.13086107, 0.3263905 , 0.13463305, 0.85270613, 0.57131058,
       0.36927538, 0.16101124, 0.75269648, 0.67722833, 0.16908194,
       0.07141343, 0.07862449, 0.15508345, 0.12366427, 0.89949794,
       0.11775408, 0.15827685, 0.11491529, 0.65334262, 0.55790969,
       0.79305264, 0.67600987, 0.2296726 , 0.37638965, 0.67415812,
       0.60162014, 0.10391125, 0.41919588, 0.51571389, 0.90759

In [ ]:
sample_submission['Survived'] = (probs.mean(axis = 1) >= 0.5)
sample_submission['Survived'] = sample_submission['Survived'].astype('int8')
sample_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
sample_submission.to_csv(os.path.join(path_root, 'submission/ensemble.csv'), index = False)